In [1]:
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from setupDataset import get_dataset
from feature_vector import get_feature_names



In [2]:
fnames = get_feature_names()

In [19]:
len(fnames)

64

# f-test, chi2, mutual information
Input: X, y  (X... die feature vectors in einem array; y... das Target passend zu X)

<hr>
per Feature:
# Wilcoxon T-Test
Input: good, bad (sprich pro Array die Samples einer Kategorie)

# Correlation Coefficient
Input: feature, target (ergo: alle Samples einer Feature Spalte, das zugehörige Target)

In [3]:
def compute_scores(filename):
    dataset = get_dataset(filename)
    X = dataset["data"]
    y = dataset["target"]
    
    f_test_scores, _ = f_classif(X, y)
    mi = mutual_info_classif(X, y)   
    chi_scores, _ = chi2(X, y)

    
    data = []
    
    for i in range(len(fnames)):
        
        indices_good = np.where(y == 1)[0]
        indices_bad = np.where(y == 0)[0]
        good = X[:, i][indices_good]
        bad  = X[:, i][indices_bad]
        ranksum, _ = stats.ranksums(good, bad) # wilcoxon
        print fnames[i]
        corr =  np.corrcoef([X[:, i], y])[1][0]
        print "corr: ", round(corr, 3), ", ranksum: ", round(ranksum, 3),", f: ", round(f_test_scores[i], 2), ",\n"
        
        
        data.append([abs(f_test_scores[i]), abs(ranksum), abs(chi_scores[i]), abs(corr), abs(mi[i])])

    
    row_label = fnames
    #print data
    col_label = ["f", "ranksum", "chi", "corrcoeff", "mi"]
    data = np.array(data)
    df = pd.DataFrame(data, index=row_label, columns=col_label)
    df.to_csv(r'abs_scores-dataset09.csv', sep='\t')
    
    
compute_scores("dataset.pickle")

as_is___exclamation_ratio
corr:  0.242 , ranksum:  5.274 , f:  64.06 ,

as_is___long_words_count
corr:  -0.413 , ranksum:  -13.748 , f:  211.88 ,

as_is___wh_pronouns
corr:  -0.231 , ranksum:  -6.945 , f:  58.41 ,

as_is___ellipsis_count
corr:  0.165 , ranksum:  3.408 , f:  28.95 ,

as_is___period_ratio
corr:  -0.152 , ranksum:  -3.224 , f:  24.34 ,

as_is___questionmark_ratio
corr:  -0.111 , ranksum:  -3.123 , f:  12.9 ,

as_is___whitespace_ratio
corr:  0.058 , ranksum:  3.188 , f:  3.46 ,

as_is___highlighters_count
corr:  -0.122 , ranksum:  -3.445 , f:  15.59 ,

as_is___paras_count
corr:  0.128 , ranksum:  2.603 , f:  17.28 ,

as_is___sent_count
corr:  -0.126 , ranksum:  -5.333 , f:  16.53 ,

as_is___word_count
corr:  -0.324 , ranksum:  -11.826 , f:  121.27 ,

as_is___ttr
corr:  0.307 , ranksum:  10.111 , f:  107.05 ,

as_is___max_wordlength
corr:  -0.215 , ranksum:  -9.55 , f:  50.24 ,

as_is___average_wordlength
corr:  -0.122 , ranksum:  -4.917 , f:  15.59 ,

as_is___median_wordle